In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 50)

In [ ]:
df_non_pol_tweets = pd.read_csv('Non_Political_Account_Tweets.csv')

In [ ]:
df_non_pol_account_info= pd.read_csv('Non_Pol_Manual_Check.csv', dtype = 'str')
df_non_pol_account_info2 = pd.read_csv('NonPol_Manual_Label2.csv', dtype = 'str')

In [ ]:
df_non_pol_account_info2.loc[df_non_pol_account_info2.User_ID == '1.61596E+18', 'User_ID'] = '1615962057249280000'

In [ ]:
df_non_pol_account_info2['User_ID'] =df_non_pol_account_info2['User_ID'].astype('str')
#df_non_pol_account_info['User_ID'] =df_non_pol_account_info['User_ID'].astype('int')


In [ ]:
#df_non_pol_account_info2.loc[df_non_pol_account_info2.User_ID == '1.61596E+18']

In [ ]:
#1.61596E+18

In [ ]:
df_non_pol_account_info2.loc[df_non_pol_account_info2.User_ID == '1.61596E+18', 'User_ID'] = '1615962057249280000'

In [ ]:
df_non_pol_account_info2['User_ID'] =df_non_pol_account_info2['User_ID'].astype('str')

In [ ]:
df_non_pol_account_info

In [ ]:
df_non_pol = df_non_pol_account_info[df_non_pol_account_info.User_ID.isin(df_non_pol_account_info2.loc[df_non_pol_account_info2.Label == 'NonPolitical'].User_ID)]

In [ ]:
df_non_pol

In [ ]:
df_non_pol =df_non_pol.drop_duplicates(subset = ['User_ID'])

In [ ]:
df_non_pol2 = pd.read_csv('Training_Political_Affiliation_Data.csv')

In [ ]:
df_non_pol2.loc[df_non_pol2.Party == 'No'].User_ID.tolist()

In [ ]:
#df_non_pol2

In [ ]:
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_non_pol3= df_account[df_account.User_ID.isin(df_non_pol2.loc[df_non_pol2.Party == 'No'].User_ID.tolist())]

In [ ]:
df_non_pol3_short = df_non_pol3[['User_ID', 'UserName', 'ScreenName','UserDescription']].rename(columns = {'UserDescription': 'User_Description'})

In [ ]:
df_non_pol3.columns

In [ ]:
#df_non_pol

In [ ]:
df_non_pol3_short['User_ID'] = df_non_pol3_short['User_ID'].astype('str')

In [ ]:
#df_non_pol3_short

In [ ]:
df_non_pol_new = pd.concat([df_non_pol, df_non_pol3_short])

In [ ]:
df_pol= df_account[df_account.User_ID.isin(df_non_pol2.loc[df_non_pol2.Party != 'No'].User_ID.tolist())]

In [ ]:
df_pol_check = df_pol.dropna(subset = ['UserDescription']).sample(200)

In [ ]:
df_pol_check= df_pol_check[['User_ID', 'UserName', 'ScreenName','UserDescription']].rename(columns = {'UserDescription': 'User_Description'})

In [ ]:
df_non_pol_new['Label'] = 0
df_pol_check['Label'] = 1

In [ ]:
len(df_pol_check)

In [ ]:
df_data = pd.concat([df_non_pol_new, df_pol_check])

In [ ]:
df_data = df_data.reset_index(drop = True)

In [ ]:
df_data = df_data.sample(frac = 1)

In [ ]:
df_data

In [ ]:
df_data['ScreenName'] = df_data['ScreenName'].str.replace('@', '')

In [ ]:
df_data = df_data[['User_ID', 'User_Description','UserName','ScreenName', 'Label']]

In [ ]:
df_data = df_data.reset_index(drop = True)

In [ ]:
df_data.dtypes

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Download NLTK stop words if not already downloaded
#nltk.download('stopwords')

# Example DataFrame

#df = pd.DataFrame(data)

# Function to remove stop words
def remove_stopwords(text):
    if isinstance(text, str):
        stop_words = set(stopwords.words('english'))
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return ' '.join(filtered_words)
    else:
        return text

# Apply the function to the DataFrame column
df_data['User_Description_Updated'] = df_data['User_Description'].apply(remove_stopwords)



In [ ]:
df_data

In [ ]:
df_data['User_Description_Updated'] = df_data['User_Description_Updated'].replace(np.nan, '')

In [ ]:
import re
def remove_links(text):
    return re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

# Apply the function to remove links from the 'Text' column
df_data['User_Description_Updated'] = df_data['User_Description_Updated'].apply(remove_links)

In [ ]:
#df_data

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
df_data['User_Description_Updated'] = df_data['User_Description_Updated'].fillna('')
#df['text_column'] = df['text_column'].fillna('')

# Initialize TfidfVectorizer with bi-gram and 50 top features
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=50)

# Fit and transform the text column
tfidf_matrix = tfidf_vectorizer.fit_transform(df_data['User_Description_Updated'])

# Get feature names
feature_names = tfidf_vectorizer.get_feature_names_out()

# Convert TF-IDF matrix to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

#print(tfidf_df)

In [ ]:
feature_names

In [ ]:
#tfidf_df.columns

In [ ]:
#tfidf_df['Label'] = df_data['Label']

In [ ]:
#tfidf_df

In [ ]:
#df_data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier()

# Define 10-fold cross-validation
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform 10-fold cross-validation
cv_scores = cross_val_score(rf_classifier, tfidf_df, df_data['Label'], cv=k_fold, scoring='f1')

# Calculate mean accuracy
mean_f1 = cv_scores.mean()
print("Mean F1 Score:", mean_f1)


In [ ]:
tfidf_df.columns

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import math

# Example DataFrame with class labels and text data
# data = {'text': ["This is an example sentence for class 0",
#                  "Another example sentence for class 0",
#                  "Yet another sentence for class 1",
#                  "Example sentence for class 1",
#                  "An example sentence for class 1"],
#         'class': [0, 0, 1, 1, 1]}

# df = pd.DataFrame(data)

# Function to extract bigrams from text
def extract_bigrams(text):
    words = text.split()
    return [words[i] + " " + words[i+1] for i in range(len(words)-1)]

# Select top 25 instances of each class
top_25_class_0 = df_data.loc[df_data['Label'] == 0]
top_25_class_1 = df_data.loc[df_data['Label'] == 1]

# Extract bigrams for each class
class_0_bigrams = [extract_bigrams(text) for text in top_25_class_0['User_Description_Updated']]
class_1_bigrams = [extract_bigrams(text) for text in top_25_class_1['User_Description_Updated']]

# Count occurrences of bigrams in each class
class_0_bigram_counts = Counter([bg for sublist in class_0_bigrams for bg in sublist])
class_1_bigram_counts = Counter([bg for sublist in class_1_bigrams for bg in sublist])

# Calculate significance score (Sbiдr am) for each bigram
significance_scores = {}
for bigram, count in class_1_bigram_counts.items():
    if bigram in class_0_bigram_counts:
        significance_scores[bigram] = math.log(count / class_0_bigram_counts[bigram])
    else:
        significance_scores[bigram] = math.log(count)

# Display the significance scores
print("Significance Scores (Sbiдr am):")
for bigram, score in significance_scores.items():
    print(f"{bigram}: {score}")


In [ ]:
#significance_scores

# User Tweet Text 

In [ ]:
df_non_political_tweets = pd.read_csv('Non_Political_Tweets.csv')

In [ ]:
df_non_political_tweets.groupby('User_ID').count()

In [ ]:
df_all_tweet.dtypes

In [ ]:
#df_all_tweet.groupby('User_ID').count()

In [ ]:
df_check_tweet = df_all_tweet[df_all_tweet.User_ID.isin(df_data.loc[df_data.Label== 1].User_ID.astype('int').tolist())]
df_check_tweet2 = df_all_tweet[df_all_tweet.User_ID.isin(df_data.loc[df_data.Label== 0].User_ID.astype('int').tolist())]

In [ ]:
df_check_tweet2

In [ ]:
#df_data.loc[df_data.Label== 1].User_ID.tolist()

In [ ]:
df_check_tweet.groupby('User_ID').count()

In [ ]:
len(df_check_tweet)

In [ ]:
len(df_non_political_tweets)

In [ ]:
def sample_tweets(group):
    if len(group) >= 20:
        return group.sample(20)
    else:
        return group

# Apply the function to the DataFrame
subset_df = df_check_tweet.groupby('User_ID', group_keys=False).apply(sample_tweets)
subset_df2 = df_check_tweet2.groupby('User_ID', group_keys=False).apply(sample_tweets)
#print(subset_df)

In [ ]:
subset_df2

In [ ]:
import pandas as pd
import re

# Function to remove mentions and links from tweet text
def remove_mentions_links(text):
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove links
    text = re.sub(r'https?://\S+', '', text)
    return text


# Apply the function to the tweet_text column
subset_df['clean_tweet_text'] = subset_df['Tweet_Text'].apply(remove_mentions_links)
subset_df2['clean_tweet_text'] = subset_df2['Tweet_Text'].apply(remove_mentions_links)
df_non_political_tweets['clean_tweet_text'] = df_non_political_tweets['text'].apply(remove_mentions_links)

In [ ]:
subset_df2

In [ ]:
#df_non_political_tweets

In [ ]:
def merge_tweets(group):
    return ' '.join(group)

# Merge tweets for each user
merged_tweets_df_non_political = df_non_political_tweets.groupby('User_ID')['clean_tweet_text'].agg(merge_tweets).reset_index()
merged_tweets_df_political = subset_df.groupby('User_ID')['clean_tweet_text'].agg(merge_tweets).reset_index()
merged_tweets_df_non_political2 = subset_df2.groupby('User_ID')['clean_tweet_text'].agg(merge_tweets).reset_index()
#print(merged_tweets_df)

In [ ]:
merged_tweets_df_non_political2

In [ ]:
merged_tweets_df_political

In [ ]:
nltk.download('stopwords')

In [ ]:
#stopwords = nltk.corpus.stopwords.words('hindi')

In [ ]:
from stopwords_hindi import hindi_sw
sw = hindi_sw.get_hindi_sw()


In [ ]:
stop_words_english = stopwords.words('english')

In [ ]:
type(list(sw))

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Additional stop words to be removed
extra_stop_words = ['RT', 'ji', 'amp']

# Function to remove stop words from tweet text
def remove_stopwords(text):
    text = text.replace('RT :', '')
    stop_words_english = stopwords.words('english')

    # Get the list of Hindi stop words
    stop_words_hindi = list(sw)

    # Combine both English and Hindi stop words
    stop_words_combined = stop_words_english + stop_words_hindi
    stop_words = set(stop_words_combined)
    # Add extra stop words
    stop_words.update(extra_stop_words)
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)


# Apply the function to the tweet_text column
merged_tweets_df_non_political['clean_tweet_text'] = merged_tweets_df_non_political['clean_tweet_text'].apply(remove_stopwords)
merged_tweets_df_non_political2['clean_tweet_text'] = merged_tweets_df_non_political2['clean_tweet_text'].apply(remove_stopwords)
merged_tweets_df_political['clean_tweet_text'] = merged_tweets_df_political['clean_tweet_text'].apply(remove_stopwords)

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

def tweet_to_base_form(tweet):
    # Convert tweet text to lowercase
    tweet = tweet.lower()
    
    # Tokenize the tweet into words
    words = word_tokenize(tweet)
    
    # Initialize WordNet lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Lemmatize each word and join them back into a string
    base_form_words = [lemmatizer.lemmatize(word) for word in words]
    base_form_tweet = ' '.join(base_form_words)
    
    return base_form_tweet


In [ ]:
merged_tweets_df_non_political['clean_tweet_text'] = merged_tweets_df_non_political['clean_tweet_text'].apply(tweet_to_base_form)
merged_tweets_df_political['clean_tweet_text'] = merged_tweets_df_political['clean_tweet_text'].apply(tweet_to_base_form)

merged_tweets_df_non_political2['clean_tweet_text'] = merged_tweets_df_non_political2['clean_tweet_text'].apply(tweet_to_base_form)


In [ ]:
merged_tweets_df_non_political['clean_tweet_text'] = merged_tweets_df_non_political['clean_tweet_text'].apply(remove_stopwords)
merged_tweets_df_non_political2['clean_tweet_text'] = merged_tweets_df_non_political2['clean_tweet_text'].apply(remove_stopwords)
merged_tweets_df_political['clean_tweet_text'] = merged_tweets_df_political['clean_tweet_text'].apply(remove_stopwords)

In [ ]:
merged_tweets_df_political['Label'] = 1

In [ ]:
merged_tweets_df_non_political['Label'] =0
merged_tweets_df_non_political2['Label'] =0

In [ ]:
df_data_tweet = pd.concat([merged_tweets_df_political[['User_ID', 'clean_tweet_text', 'Label']],merged_tweets_df_non_political[['User_ID', 'clean_tweet_text', 'Label']], merged_tweets_df_non_political2[['User_ID', 'clean_tweet_text', 'Label']]])

In [ ]:
df_data_tweet=df_data_tweet.sample(frac = 1).reset_index(drop = True)

In [ ]:
#df_data_tweet.groupby('Label').count()

In [ ]:
keywords_list = ['bjp', 'inc', 'modi', 'rahul', 'gandhi', 'yogi', 'congress', 'aap', 'kejriwal', 
              'भाजपा','भारतीय राष्ट्रीय कांग्रेस', 'मोदी' ,'राहुल','गांधी' ,'योगी','कांग्रेस' ,'आप', 'केजरीवाल' ]

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
df_data['User_Description_Updated'] = df_data['User_Description_Updated'].fillna('')
#df['text_column'] = df['text_column'].fillna('')

# Initialize TfidfVectorizer with bi-gram and 50 top features
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=25)

# Fit and transform the text column
tfidf_matrix = tfidf_vectorizer.fit_transform(df_data['User_Description_Updated'])

# Get feature names
feature_names = tfidf_vectorizer.get_feature_names_out()

# Convert TF-IDF matrix to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

#print(tfidf_df)

In [ ]:
feature_names

In [ ]:
df_data_tweet['clean_tweet_text'] = df_data_tweet['clean_tweet_text'].fillna('')

# Initialize TfidfVectorizer with bi-gram and 50 top features
tfidf_vectorizer_tweet = TfidfVectorizer(ngram_range=(1, 3), max_features=25)

# Fit and transform the text column
tfidf_matrix_tweet = tfidf_vectorizer_tweet.fit_transform(df_data_tweet['clean_tweet_text'])

# Get feature names
feature_names_tweet = tfidf_vectorizer_tweet.get_feature_names_out()

# Convert TF-IDF matrix to DataFrame
tfidf_df_tweet = pd.DataFrame(tfidf_matrix_tweet.toarray(), columns=feature_names_tweet)

In [ ]:
feature_names_tweet

In [ ]:
set(feature_names_tweet)

In [ ]:
tfidf_df['User_ID'] = df_data['User_ID'].astype('str')
tfidf_df_tweet['User_ID'] = df_data_tweet['User_ID'].astype('str')
df_data_small = df_data[['User_ID', 'Label']]
df_data_small['User_ID'] = df_data_small['User_ID'].astype('str')

In [ ]:
tf_idf_data = pd.merge(tfidf_df, tfidf_df_tweet, on = 'User_ID', how = 'left')
tf_idf_data = pd.merge(tf_idf_data, df_data_small[['User_ID', 'Label']])

In [ ]:
tf_idf_data = tf_idf_data.replace(np.nan, 0)

In [ ]:
tf_idf_data.columns

In [ ]:
tf_idf_data.User_ID = tf_idf_data.User_ID.astype('int')

In [ ]:
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')
df_non_account = pd.read_csv('Non_Political_Info.csv')

In [ ]:
df_account[['User_ID', 'AveragePosts']]

In [ ]:
#pd.to_datetime(df_non_account['date'])

In [ ]:
pd.Timestamp.now(tz='UTC')

In [ ]:
df_non_account['date'] = pd.to_datetime(df_non_account['Created_At'])
df_non_account['date'] = df_non_account['date'].dt.strftime('%b %d, %Y')
df_non_account['date'] = pd.to_datetime(df_non_account['date']).dt.tz_localize('UTC')
df_non_account['Account_Age'] = (pd.Timestamp.now(tz='UTC') - df_non_account['date']).dt.days
df_non_account['AveragePosts'] = df_non_account.Post_Count/ df_non_account.Account_Age

In [ ]:
#df_non_account

In [ ]:
df_non_account['AveragePosts'] = df_non_account.Post_Count/ df_non_account.Account_Age

In [ ]:
df_non_account[['User_ID','AveragePosts']]

In [ ]:
df_acc_small_check = pd.concat([df_account[['User_ID', 'AveragePosts']], df_non_account[['User_ID','AveragePosts']]])

In [ ]:
df_acc_small_check

In [ ]:
tf_idf_data = pd.merge(tf_idf_data, df_acc_small_check)

In [ ]:
class_label = tf_idf_data['Label']
train_data = tf_idf_data.drop(columns = ['User_ID', 'Label'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


class_label = tf_idf_data['Label']
train_data = tf_idf_data.drop(columns = ['User_ID', 'Label'])


# Initialize Random Forest Classifier
rf_classifier2 = RandomForestClassifier()

# Define 10-fold cross-validation
k_fold2 = KFold(n_splits=10)

# Perform 10-fold cross-validation
cv_scores2_f1 = cross_val_score(rf_classifier2, train_data, class_label, cv=k_fold2, scoring='f1')
cv_scores2_pr = cross_val_score(rf_classifier2, train_data, class_label, cv=k_fold2, scoring='precision')
cv_scores2_re = cross_val_score(rf_classifier2, train_data, class_label, cv=k_fold2, scoring='recall')

# Calculate mean accuracy
mean2_f1 = cv_scores2_f1.mean()
print("Mean F1 Score:", mean2_f1)
print("Mean PR Score:", cv_scores2_pr.mean())
print("Mean RE Score:",cv_scores2_re.mean())

In [ ]:
#tf_idf_data.groupby('Label').count()

# Account Political Affiliation

In [ ]:
len(train_data)

In [ ]:
rf_classifier2.fit(train_data, class_label)

In [ ]:
feature_importances = rf_classifier2.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': train_data.columns, 'Importance': feature_importances})
feature_importance_df.sort_values('Importance', ascending = False).iloc[0:20]

In [ ]:
df_non_pol_following = pd.read_csv('Non_Political_Following.csv')

In [ ]:
df_niva_duck = pd.read_csv('NivaDuck.csv')

In [ ]:
df_niva_duck_small = df_niva_duck[['id', 'party']].rename(columns = {'id': 'Following_ID'})

In [ ]:
df_non_pol_follow_pol = pd.merge(df_non_pol_following, df_niva_duck_small, how = 'left')

In [ ]:
#df_non_pol_follow_pol['Political'] = 'True'

In [ ]:
df_non_pol_follow_pol

In [ ]:
df_non_pol_follow_pol.groupby('User_ID').count()

In [ ]:
import pandas as pd


# Group by 'User ID' and 'Political Label' and count occurrences
grouped = df_non_pol_follow_pol.groupby(['User_ID', 'party']).size().reset_index(name='Frequency')

# Pivot the table to have User ID as index and Political Label as columns with frequency counts
pivot_table = grouped.pivot_table(index='User_ID', columns='party', values='Frequency', fill_value=0)

# If a user doesn't have a particular label, fill it with 0
pivot_table = pivot_table.reindex(columns=['AAP', 'BJP', 'INC', 'Other'], fill_value=0)

#print(pivot_table)


In [ ]:
df_non_pol_follow_pol = df_non_pol_follow_pol.groupby('User_ID').count().reset_index()[['User_ID', 'Following_ID']].rename(columns = {'Following_ID': 'Total_Following'})

In [ ]:
pivot_new = pd.merge(pivot_table.reset_index(), df_non_pol_follow_pol, how = 'right')

In [ ]:
pivot_new['Political'] = pivot_new['AAP'] + pivot_new['BJP'] + pivot_new['INC'] + pivot_new['Other']

In [ ]:
pivot_new['Political_Ratio'] = pivot_new['Political']/pivot_new['Total_Following']

In [ ]:
pivot_new = pivot_new.replace(np.nan, 0)

In [ ]:
pivot_new[['User_ID', 'Political_Ratio']]

In [ ]:
df_political_following = pd.read_csv('Cluster_Following_Data2.csv')

In [ ]:
df_following_info_new = pd.concat([df_political_following[['User_ID', 'Political_Ratio']], pivot_new[['User_ID', 'Political_Ratio', 'AAP', 'BJP', 'INC', 'Other']]])

In [ ]:
tf_idf_data = pd.merge(tf_idf_data, df_following_info_new, how = 'left')

In [ ]:
#df_political_following

In [ ]:
tf_idf_data = tf_idf_data.replace(np.nan, 0)

In [ ]:
tf_idf_data[['User_ID', 'Political_Ratio', 'AAP', 'BJP', 'INC','Other']].to_csv('Cluster_Following_Data3.csv', index = False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


class_label = tf_idf_data['Label']
train_data = tf_idf_data.drop(columns = ['User_ID', 'Label'])


# Initialize Random Forest Classifier
rf_classifier2 = RandomForestClassifier()

# Define 10-fold cross-validation
k_fold2 = KFold(n_splits=10)

# Perform 10-fold cross-validation
cv_scores2_f1 = cross_val_score(rf_classifier2, train_data, class_label, cv=k_fold2, scoring='f1')
cv_scores2_pr = cross_val_score(rf_classifier2, train_data, class_label, cv=k_fold2, scoring='precision')
cv_scores2_re = cross_val_score(rf_classifier2, train_data, class_label, cv=k_fold2, scoring='recall')

# Calculate mean accuracy
mean2_f1 = cv_scores2_f1.mean()
print("Mean F1 Score:", mean2_f1)
print("Mean PR Score:", cv_scores2_pr.mean())
print("Mean RE Score:",cv_scores2_re.mean())

In [ ]:
tf_idf_data.to_csv('Political_NonPolitical_Model_Data.csv', index = False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


# political_labels_up = {
#     'AAP': 2,
#     'ABVP': 1,
#     'BJP': 1,
#     'DMK': 2,
#     'GOV' : 2,
#     'INC' : 0,
#     'No' : 2,
#     'RJD' : 0,
#     'SP': 0,
#     'RSS' : 1,
#     'SS' : 1,
#     'GOV':2,
#     'Undetermined' : 2,
#     'VHP': 1
# }


political_labels_up = {
    'AAP': 2,
    'ABVP': 1,
    'BJP': 1,
    'DMK': 2,
    'GOV' : 2,
    'INC' : 0,
    'No' : 3,
    'RJD' : 0,
    'SP': 0,
    'RSS' : 1,
    'SS' : 1,
    'GOV':2,
    'Undetermined' : 3,
    'VHP': 1
}


y = tf_idf_data['Label']
X = tf_idf_data.drop(columns = ['User_ID', 'Label'])


scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

# df_new_short6= df_new_short6.reset_index(drop = True)
# # Split the data into features (X) and labels (y)
# X = df_new_short6.drop(columns=['Party_Label', 'Party', 'User_ID', 'UserFollowers', 
#                                 'UserFriends', 'UserFavoriteCount', 'TotalStatus',
#                                 'AccountAgeinDays', 'AveragePosts', 'AverageFavoriteCount',
#                                 '%_of_Retweet_Collected', '%_of_Tweet_Collected', 'Total_Hashtags',
#                                 'Retweet_Count', 'Favorite_Count', 'Average_Engagement_on_Tweets',
#                                 'Average_Engagement_on_Retweets', 'Tweet_Max_Engagement',
#                                 'Tweet_Max_Retweet', 'Tweet_Max_Favorite', 'Tweet_Max_Hashtags',
#                                 'h-index', 'Total_Unique_Source', 'followers_friends_ratio',
#                                 'tweet_retweet_ratio', 'h-index_Total'])

# y = df_new_short6['Party_Label']

#X2 = (X - X.min()) / (X.max() - X.min())

# Define a logistic regression model
model =  RandomForestClassifier()
scorer = make_scorer(f1_score)

# Perform 10-fold cross-validation
scores = cross_val_score(model, X, y, cv=10, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores:", scores)

# Calculate and print the mean cross-validation score
print("Mean cross-validation score:", scores.mean())

In [ ]:
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have loaded your dataset into X and y
f1_scores = []
precision_scores = []
recall_scores = []



# Now X_train_resampled and y_train_resampled contain the oversampled data


# Assuming you have defined X (features) and y (labels) and have imported necessary modules

# Initialize StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=False)

# Initialize lists to store predictions and true labels
all_preds = []
all_labels = []

# Iterate over each fold
for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]


    # Perform oversampling using SMOTE
    #smote = SMOTE( random_state=42)
    #X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    model =  RandomForestClassifier()
    model.fit(X_train, y_train)
    
    # Make predictions
    preds = model.predict(X_test)
    
    f1 = f1_score(y_test, preds, average='weighted')
    precision = precision_score(y_test, preds, average='weighted')
    recall = recall_score(y_test, preds, average='weighted')
    # Here, you would typically use your fine-tuned model to make predictions on X_test
    # For demonstration, let's assume we have random predictions
    #preds = np.random.choice([0, 1], size=len(y_test))  # Replace this with your model predictions
    f1_scores.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)

    # Print evaluation metrics for this fold
    print("Fold F1 Score:", f1)
    print("Fold Precision:", precision)
    print("Fold Recall:", recall)
    

    # Append predictions and true labels
    all_preds.extend(preds)
    all_labels.extend(y_test)

# Compute confusion matrix for all predictions and true labels
conf_matrix = confusion_matrix(all_labels, all_preds)

# Display confusion matrix
print("Confusion Matrix:")
print(conf_matrix)


mean_f1 = np.mean(f1_scores)
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)

# Print mean values of evaluation metrics
print("Mean F1 Score:", mean_f1)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)

In [ ]:
# Convert the confusion matrix to numpy array
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
confusion_matrix = np.array(conf_matrix)

true_positives = np.diag(confusion_matrix)
false_positives = np.sum(confusion_matrix, axis=0) - true_positives
false_negatives = np.sum(confusion_matrix, axis=1) - true_positives

# Calculate precision, recall, and F1 score for each class
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print precision, recall, and F1 score for each class
for i in range(len(precision)):
    print(f"Class {i}:")
    print(f"  Precision: {precision[i]}")
    print(f"  Recall: {recall[i]}")
    print(f"  F1 Score: {f1_score[i]}")